Подготовка датасета

In [23]:
import os

import librosa
import pandas as pd

# Пути к файлам
dataset_path = "data/test/normal_voices"
xlsx_path = "data/test/Speeches.xlsx"

# Читаем данные из Excel
df = pd.read_excel(xlsx_path)

# Загружаем аудиофайлы и их MFCC
audio_data = {}
for _, row in df.iterrows():
    wav_file = row["file"]
    text = row["text"]
    audio_path = os.path.join(dataset_path, f"{wav_file}.wav")

    if os.path.exists(audio_path):
        y, sr = librosa.load(audio_path, sr=22050)
        mfcc = librosa.feature.mfcc(y=y, sr=sr)
        audio_data[text] = (y, sr, mfcc)

/home/makarov/miniconda3/lib/python3.12/site-packages/librosa/core/spectrum.py:266: UserWarning: n_fft=2048 is too large for input signal of length=0
  warnings.warn(


Графовая модель на основе DTW

In [29]:

from fastdtw import fastdtw

# Функция поиска ближайшего совпадения
def find_closest_match(text_mfcc, audio_data):
    best_match = None
    best_dist = float("inf")

    for text, (y, sr, mfcc) in audio_data.items():
        dist, _ = fastdtw(text_mfcc.T, mfcc.T)
        if dist < best_dist:
            best_dist = dist
            best_match = y

    return best_match

# Генерация речи
def generate_speech(text, audio_data):
    # Если текст уже есть в аудиоданных, возвращаем его напрямую
    if text in audio_data:
        speech, sr, _ = audio_data[text]
    else:
        # Иначе ищем наиболее похожий фрагмент
        ref_text = list(audio_data.keys())[0]  # Выбираем любой реальный образец
        ref_audio, ref_sr, ref_mfcc = audio_data[ref_text]

        # Берем MFCC случайного реального аудиофайла (чтобы избежать случайного шума)
        text_mfcc = ref_mfcc
        speech = find_closest_match(text_mfcc, audio_data)

    # Сохраняем результат
    sf.write("output_dtw.wav", speech, 22050)

# Пример вызова
generate_speech("Привет общество", audio_data)


WaveNet с обучением с нуля

In [27]:
import torch
import torch.nn as nn
import torch.optim as optim
import librosa
import os
import pandas as pd

# Пути к данным
dataset_path = "data/test/normal_voices"
xlsx_path = "data/test/Speeches.xlsx"

# Читаем Excel-файл
df = pd.read_excel(xlsx_path)

# Модель WaveNet
class WaveNet(nn.Module):
    def __init__(self, n_input, n_output):
        super(WaveNet, self).__init__()
        # Первый слой: 128 каналов на входе, 32 канала на выходе
        self.conv1 = nn.Conv1d(n_input, 32, kernel_size=3, padding=1)
        self.relu = nn.ReLU()
        # Второй слой: 32 канала на входе, n_output на выходе
        self.conv2 = nn.Conv1d(32, n_output, kernel_size=3, padding=1)

    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.conv2(x)
        return x

# Генерация спектрограммы
def audio_to_mel(audio_path):
    y, sr = librosa.load(audio_path, sr=22050)
    mel = librosa.feature.melspectrogram(y=y, sr=sr)
    return mel

# Обучение модели
def train_wavenet(dataset_path, df):
    n_input = 128  # Размерность Mel-спектрограммы (каналы)
    n_output = 128  # Размерность выходных данных

    model = WaveNet(n_input=n_input, n_output=n_output)
    optimizer = optim.Adam(model.parameters())
    criterion = nn.MSELoss()

    for epoch in range(10):
        for _, row in df.iterrows():
            audio_path = os.path.join(dataset_path, f"{row['file']}.wav")

            if not os.path.exists(audio_path):
                continue  # Пропустить, если файл не найден

            mel = audio_to_mel(audio_path)
            mel = torch.tensor(mel).unsqueeze(0).float()  # Добавляем batch dimension: (1, freq, time)

            # Трансформируем в правильную форму: (batch_size, channels, time)
            mel = mel.squeeze(0)  # Убираем лишнюю ось, теперь (freq, time)
            mel = mel.unsqueeze(0)  # Добавляем ось batch size: (1, freq, time)

            output = model(mel)

            loss = criterion(output, mel)  # Выход должен совпадать с входом
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        print(f"Epoch {epoch}, Loss: {loss.item()}")
    return model

# Запуск обучения
model = train_wavenet(dataset_path, df)


/home/makarov/miniconda3/lib/python3.12/site-packages/librosa/core/spectrum.py:266: UserWarning: n_fft=2048 is too large for input signal of length=0
  warnings.warn(


Epoch 0, Loss: 6.127837181091309
Epoch 1, Loss: 3.5629818439483643
Epoch 2, Loss: 2.5861129760742188
Epoch 3, Loss: 2.2238385677337646
Epoch 4, Loss: 2.280803918838501
Epoch 5, Loss: 2.673626184463501
Epoch 6, Loss: 2.1938414573669434
Epoch 7, Loss: 2.382476329803467
Epoch 8, Loss: 1.68925940990448
Epoch 9, Loss: 1.6742852926254272


In [28]:
import torch
import librosa
import numpy as np
import soundfile as sf

# Преобразование текста в мел-спектрограмму (для упрощения)
def text_to_mel_spectrogram(text):
    return np.random.random((128, 50))  # Заглушка для генерации случайной спектрограммы

# Используем WaveNet для генерации аудио
def text_to_audio(text, model, sample_rate=22050):
    mel_spectrogram = text_to_mel_spectrogram(text)  # Преобразуем текст в мел-спектрограмму
    mel_tensor = torch.tensor(mel_spectrogram).unsqueeze(0).float()  # Преобразуем в тензор
    output = model(mel_tensor)  # Генерируем выход модели

    audio = mel_to_audio(output.squeeze(0).detach().numpy(), sample_rate)  # Преобразуем спектрограмму в аудио
    return audio

# Преобразование мел-спектрограммы в аудио
def mel_to_audio(mel, sample_rate=22050):
    return librosa.feature.inverse.mel_to_audio(mel, sr=sample_rate)

# Сохранение аудио в файл
def save_audio(audio, filename, sample_rate=22050):
    sf.write(filename, audio, sample_rate)

# Пример использования
text = "Hello, this is a test."
audio = text_to_audio(text, model)
save_audio(audio, "output_audio.wav")


 LSTM + Spectrogram-based

In [ ]:
import torch
import torch.nn as nn
import librosa
import numpy as np
import pandas as pd
import soundfile as sf

# Модель LSTM для преобразования текста в спектрограмму
class TextToSpectrogramLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(TextToSpectrogramLSTM, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size)
        # Изменяем размерность на выходе с 128 на 256, чтобы соответствовать размеру мел-спектрограммы
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        out, _ = self.lstm(x)
        out = self.fc(out)
        return out

# Функция для преобразования аудио в мел-спектрограмму
def audio_to_mel(audio_path):
    # Загрузка аудиофайла
    y, sr = librosa.load(audio_path, sr=None)

    # Вычисление мел-спектрограммы с 256 мел-частотными бинами
    mel_spectrogram = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=256)
    mel_spectrogram = librosa.power_to_db(mel_spectrogram, ref=np.max)

    # Транспонируем, чтобы размерность была (время, признаки)
    return mel_spectrogram.T

# Пример тренировки модели
def train_lstm(dataset_path, df):
    model = TextToSpectrogramLSTM(input_size=256, hidden_size=512, output_size=256)  # Устанавливаем output_size=256
    optimizer = torch.optim.Adam(model.parameters())
    criterion = nn.MSELoss()

    # Тренировка
    for epoch in range(10):  # Количество эпох
        for _, row in df.iterrows():
            wav_file = f'data/test/normal_voices/{row["file"]}.wav'  # Читаем путь к файлу
            text = row['text']  # Текст (пока не используется, но можно добавить)

            # Преобразование аудио в спектрограмму
            mel = audio_to_mel(wav_file)
            mel = torch.tensor(mel).unsqueeze(0).float()  # добавляем размерность для батча

            # Прогон через модель
            output = model(mel)
            loss = criterion(output, mel)  # Сравниваем с мел-спектрограммой

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        print(f"Epoch {epoch}, Loss: {loss.item()}")


# Запуск тренировки
train_lstm(dataset_path, df)


/home/makarov/miniconda3/lib/python3.12/site-packages/librosa/core/spectrum.py:266: UserWarning: n_fft=2048 is too large for input signal of length=0
  warnings.warn(
